# <center> <span style="color:red">Music recognition </span> </center>

The purpose of this notebook is to take in music spectrograms and to recognize musical genres

## Import 

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

## Import Model Linear and MLP 

In [2]:
import sys  
sys.path.insert(0, '../../python')

from linearModel import linearModel
from mlp import MLPModel

## Create Model 

In [3]:
lm = linearModel()
mlp = MLPModel()

## Algorithme 

In [4]:
def import_images_and_assign_labels(
        folder, label, X, Y):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8, 8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)

In [5]:
def import_dataset():
    dataset_folder = "G:/Programmes/Python/projetAnnuel/data"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    import_images_and_assign_labels(
        os.path.join(train_folder, "Electro"), 1.0, X_train, y_train
    )
    import_images_and_assign_labels(
        os.path.join(train_folder, "Metal"), -1.0, X_train, y_train
    )
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "Electro"), -1.0, X_test, y_test
    )
    import_images_and_assign_labels(
        os.path.join(test_folder, "Metal"), 1.0, X_test, y_test
    )

    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))

In [6]:
def display_value(number,array):
    print("Sur le dataset de Train")
    cpt = 0
    for i in array:
        print(cpt," ",i)
        cpt+=1
        if(cpt==number):
            cpt=0

In [7]:
def run_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()

    
    resultat1 = mlp.create_mlp_model([2,4,4,1])
    model1 = resultat1[0]
       
    
    flattened_dataset_inputs = []
    for p in X_train:
        flattened_dataset_inputs.append(p[0])
        flattened_dataset_inputs.append(p[1])
        
    print("flattened_dataset_inputs",flattened_dataset_inputs)
    
    print("X_train",X_train[0])
    print("type X_train",type(X_train))
    print("Dimension X_train",X_train.shape)
    print("\n")
    mlp.train_classification_stochastic_gradient_backpropagation(model1,
                                                                flattened_dataset_inputs,
                                                                y_train,
                                                                alpha=0.01,
                                                                iterations_count=5000)

    predicted_outputs = [mlp.predict_mlp_model_classification(model1, p) for p in X_train]
       
    display_value(160,predicted_outputs)
    
    mlp.free_MLP(resultat1)

In [8]:
def run_linear():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat2 = lm.create_linear_model(2)
    model2 = resultat2[0]


    flattened_dataset_inputs = []
    for p in X_train:
        flattened_dataset_inputs.append(p[0])
        flattened_dataset_inputs.append(p[1])

    lm.train_classification_rosenblatt_rule_linear_model(model2, flattened_dataset_inputs,
                                                            y_train, 0.01,100000)

    predicted_outputs = [lm.predict_linear_model_classification(model2, p) for p in X_train]
    
    display_value(160,predicted_outputs)
    
    lm.destroy_linear_model(resultat2)

In [9]:
run_mlp()

flattened_dataset_inputs [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.333333333333

In [10]:
run_linear()

Sur le dataset de Train
0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
5   -1.0
6   -1.0
7   -1.0
8   -1.0
9   -1.0
10   -1.0
11   -1.0
12   -1.0
13   -1.0
14   -1.0
15   -1.0
16   -1.0
17   -1.0
18   -1.0
19   -1.0
20   -1.0
21   -1.0
22   -1.0
23   -1.0
24   -1.0
25   -1.0
26   -1.0
27   -1.0
28   -1.0
29   -1.0
30   -1.0
31   -1.0
32   -1.0
33   -1.0
34   -1.0
35   -1.0
36   -1.0
37   -1.0
38   -1.0
39   -1.0
40   -1.0
41   -1.0
42   -1.0
43   -1.0
44   -1.0
45   -1.0
46   -1.0
47   -1.0
48   -1.0
49   -1.0
50   -1.0
51   -1.0
52   -1.0
53   -1.0
54   -1.0
55   -1.0
56   -1.0
57   -1.0
58   -1.0
59   -1.0
60   -1.0
61   -1.0
62   -1.0
63   -1.0
64   -1.0
65   -1.0
66   -1.0
67   -1.0
68   -1.0
69   -1.0
70   -1.0
71   -1.0
72   -1.0
73   -1.0
74   -1.0
75   -1.0
76   -1.0
77   -1.0
78   -1.0
79   -1.0
80   -1.0
81   -1.0
82   -1.0
83   -1.0
84   -1.0
85   -1.0
86   -1.0
87   -1.0
88   -1.0
89   -1.0
90   -1.0
91   -1.0
92   -1.0
93   -1.0
94   -1.0
95   -1.0
96   -1.0
97   -1.0
98   -